### Projeto de avaliação da disciplina Introdução à Análise de Dados - Prof. Luciano Barbosa
#### Alunos: David Soares (dsm3@cin.ufpe.br) & Wanderson R. Marques (wrm@cin.ufpe.br)

#### Importação das Bibliotecas e do Dataset

In [1]:
import pandas as pd
import numpy as np

In [16]:
# Importação dataset
df = pd.read_csv('Focos_2021-01-01_2021-08-31.csv')

In [17]:
# Amostra do dataset
df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2021/01/01 17:00:00,AQUA_M-T,Brasil,PARA,SANTA MARIA DAS BARREIRAS,Amazonia,NaN,NaN,NaN,-8.577,-49.952,8.8
1,2021/01/01 17:05:00,AQUA_M-T,Brasil,PARA,AVEIRO,Amazonia,NaN,NaN,NaN,-3.909,-56.037,18.9
2,2021/01/01 17:05:00,AQUA_M-T,Brasil,PARA,AVEIRO,Amazonia,NaN,NaN,NaN,-3.904,-56.045,9.5
3,2021/01/01 17:05:00,AQUA_M-T,Brasil,PARA,JACAREACANGA,Amazonia,NaN,NaN,NaN,-5.845,-57.716,149.8
4,2021/01/01 17:05:00,AQUA_M-T,Brasil,PARA,ORIXIMINA,Amazonia,NaN,NaN,NaN,-1.587,-55.922,11.3


### Pré-processamento dos Dados

#### Definição de Tipos

In [18]:
# Visualização dos tipos
df.dtypes

datahora         object
satelite         object
pais             object
estado           object
municipio        object
bioma            object
diasemchuva     float64
precipitacao    float64
riscofogo       float64
latitude        float64
longitude       float64
frp             float64
dtype: object

In [19]:
# Ajuste dos Tipos
df['datahora'] = pd.to_datetime(df['datahora'], format="%Y/%m/%d %H:%M:%S")
df['satelite'] = df['satelite'].astype('category')
df['pais'] = df['pais'].astype('category')
df['estado'] = df['estado'].astype('category')
df['municipio'] = df['municipio'].astype('category')
df['bioma'] = df['bioma'].astype('category')

In [20]:
# Verificação dos tipos após modificação
df.dtypes

datahora        datetime64[ns]
satelite              category
pais                  category
estado                category
municipio             category
bioma                 category
diasemchuva            float64
precipitacao           float64
riscofogo              float64
latitude               float64
longitude              float64
frp                    float64
dtype: object